Segment-Anything

Generate mask for an given image using Segmenting Anything library.
https://github.com/facebookresearch/segment-anything
Output an numpy datafile in specified output directory.

In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python
!pip install torch
!pip install torchvision

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\colin\appdata\local\temp\pip-req-build-dzviiqsr
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\colin\AppData\Local\Temp\pip-req-build-dzviiqsr'


In [2]:
import logging
import os
import sys
import re
import cv2
import numpy as np
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator

In [3]:
# model type
model_type = os.environ.get("model_type", "vit_h") # default, vit_h, vit_l or vit_b

# checkpoint
# different model type requires different check point
checkpoint_path = os.environ.get("checkpoint_path")

# input image
input_image_path = os.environ.get("input_image_path")

# temporal data storage for local execution
data_dir = os.environ.get('data_dir', '../../data/')

# dummy_output (to be fixed once C3 supports < 1 outputs)
# output_dummy = os.environ.get('output_dummy', 'output_dummy')

In [4]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [5]:
# get masks from a given prompt
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
input_image = cv2.imread(input_image_path)

# generate masks for image
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(input_image)


# save generated masks into given directory
output_directory, output_filename = os.path.split(data_dir)
os.makedirs(output_directory, exist_ok=True)
np.save(data_dir + "generate_masks", masks)
